In [9]:
import math
import matplotlib as mpl
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display
import json
import base64
import os
import sys

# Smoothed Armchair Graphene Nanoribbon Junctions Explorer

As Moore's law approaches its fundamental limits, the development of nanoelectronic devices using low-dimension materials has become a promising avenue for further miniaturization and performance improvements. Among the various novel materials, graphene nanoribbons (GNRs) have emerged as particularly attractive candidates due to their unique electronic properties. However, the design of efficient nanoelectronic components with a minimal spatial footprint remains a significant challenge. This tool allows to explore a wide dataset of GNR junctions and investigate various strategies for designing optimal-sized interconnects. For each set of parameters, the tool will provide various informations about the junction's electronic properties. All the raw data can then be downloaded for further analysis.

If you use this tool, please cite the following works:

* **J. Leuenberger, K. Cernevics, O. V. Yazyev, Optimizing Nanoelectronics Footprint by Rounding Graphene Nanoribbon Junctions**
* **J. Leuenberger, K. Cernevics, O. V. Yazyev, Smoothed Armchair Graphene Nanoribbon Junctions Explorer (SAJEx), doi:10.tbd/tbd**

In [10]:
path = "./data" if os.path.isdir("./data") else "/data"
# print(f"Path: {path}")

In [11]:
def get_wext_max(N_value):
    security = 1
    return math.floor(N_value / 2) - 1 - security

def get_N_min(wext_value):
    for N in range(20, 80+1, 3):
        if get_wext_max(N) >= wext_value:
            return N

In [12]:
shape = widgets.Dropdown(options=['armchair', 'round', 'zigzag'], value='armchair', description='shape')
N = widgets.IntSlider(min=20, max=80, step=3, description="N")
wext = widgets.IntSlider(min=0, max=get_wext_max(N.value), step=1, description="wₑₓₜ")
a_ratio = widgets.FloatText(value=-1, description='Armchair ratio [%]', disabled=True)
z_ratio = widgets.FloatText(value=-1, description='Zigzag ratio [%]', disabled=True)
tau_around_0p00_ev = widgets.FloatText(value=-1, description='τ @ 0.00 eV', disabled=True)
tau_around_0p01_ev = widgets.FloatText(value=-1, description='τ @ 0.01 eV', disabled=True)
tau_around_0p10_ev = widgets.FloatText(value=-1, description='τ @ 0.10 eV', disabled=True)

html_button_summary = widgets.HTML(value="""<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="" href="data:text/json;base64," download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button" disabled>-</button>
</a>
</body>
</html>
""")

html_button_conductance = widgets.HTML(value="""<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="" href="data:text/json;base64," download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button" disabled>-</button>
</a>
</body>
</html>
""")

html_button_vertices = widgets.HTML(value="""<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="" href="data:text/json;base64," download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button" disabled>-</button>
</a>
</body>
</html>
""")


html_button_edges = widgets.HTML(value="""<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="" href="data:text/json;base64," download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button" disabled>-</button>
</a>
</body>
</html>
""")

In [13]:
def update_image(shape, N, r):
    fig, axs = plt.subplots(ncols=4, nrows=1, figsize=(19.2, 4.8), dpi=100)
    try:
        axs[0].imshow(mpl.image.imread(f"{path}/shape={shape}/N={N}/wext={r}/edge.png"))
        axs[1].imshow(mpl.image.imread(f"{path}/shape={shape}/N={N}/wext={r}/conductance.png"))
        axs[2].imshow(mpl.image.imread(f"{path}/shape={shape}/N={N}/wext={r}/ldos.png"))
        axs[3].imshow(mpl.image.imread(f"{path}/shape={shape}/N={N}/wext={r}/current.png"))
        for ax in axs.ravel():
            ax.axis("off")
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_anchor('S')
        plt.subplots_adjust(wspace=-0.05, hspace=0)
    except Exception as e:
        print(e)
        # plt.close()
        # print(f"Missing data !")
        pass
    
def update_wext_max(*args):
    wext_current = wext.value
    wext_max = get_wext_max(N.value)
    if wext_current > wext_max:
        wext.value = wext_max
    wext.max = wext_max
    
def update_N_min(*args):
    N_current = N.value
    N_min = get_N_min(wext.value)
    if N_current < N_min:
        N.value = N_min
    N.min = N_min

def update_texts(change):
    try:
        infos = json.load(open(f"{path}/shape={shape.value}/N={N.value}/wext={wext.value}/summary.json"))
        a_ratio.value = infos["armchair_ratio"]
        z_ratio.value = infos["zigzag_ratio"]
        tau_around_0p00_ev.value = infos["tau_around_0p00_ev"]
        tau_around_0p01_ev.value = infos["tau_around_0p01_ev"]
        tau_around_0p10_ev.value = infos["tau_around_0p10_ev"]
    except Exception as e:
        print(e)
        pass

def update_downloadbutton(*args):
    
    base_html = """<html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    </head>
    <body>
    <a download="FILENAME" href="data:text/json;base64,PAYLOAD" download>
    <button class="p-Widget jupyter-widgets jupyter-button widget-button">TEXT</button>
    </a>
    </body>
    </html>"""

    res = open(f"{path}/shape={shape.value}/N={N.value}/wext={wext.value}/summary.json").read()
    b64 = base64.b64encode(res.encode())
    payload = b64.decode().replace("\"", "`")
    filename = f"summary_shape={shape.value}_N={N.value}_wext={wext.value}.json"
    html_button_summary.value = base_html.replace("FILENAME", filename).replace("PAYLOAD", payload).replace("TEXT", f"Summary (JSON, {sys.getsizeof(payload)/1024:.1f} ko)")

    res = open(f"{path}/shape={shape.value}/N={N.value}/wext={wext.value}/conductance.csv").read()
    b64 = base64.b64encode(res.encode())
    payload = b64.decode().replace("\"", "`")
    filename = f"conductance_shape={shape.value}_N={N.value}_wext={wext.value}.csv"
    html_button_conductance.value = base_html.replace("FILENAME", filename).replace("PAYLOAD", payload).replace("TEXT", f"Conductance (CSV, {sys.getsizeof(payload)/1024:.1f} ko)")

    res = open(f"{path}/shape={shape.value}/N={N.value}/wext={wext.value}/vertices.csv").read()
    b64 = base64.b64encode(res.encode())
    payload = b64.decode().replace("\"", "`")
    filename = f"vertices_shape={shape.value}_N={N.value}_wext={wext.value}.csv"
    html_button_vertices.value = base_html.replace("FILENAME", filename).replace("PAYLOAD", payload).replace("TEXT", f"Vertices (CSV, {sys.getsizeof(payload)/1024:.1f} ko)")

    res = open(f"{path}/shape={shape.value}/N={N.value}/wext={wext.value}/edges.csv").read()
    b64 = base64.b64encode(res.encode())
    payload = b64.decode().replace("\"", "`")
    filename = f"edges_shape={shape.value}_N={N.value}_wext={wext.value}.csv"
    html_button_edges.value = base_html.replace("FILENAME", filename).replace("PAYLOAD", payload).replace("TEXT", f"Edges (CSV, {sys.getsizeof(payload)/1024:.1f} ko)")


## Inputs

In order to investigate the strategies for designing a GNR junction, the following parameters can be chosen:

* **shape** corresponds to the junction's tip as material is removed. This can either be "armchair" which maximises the amount of armchair edges, "zigzag" which maximises the amount of zigzag edges, or "round" which is an in-between.
* **N** is the width of the GNRs, in number of atom layers. Allowed values are **N** $\in[5, 80]$. Note that only metallic GNRs are considered, i.e. **N** $=3n+2$.
* **wₑₓₜ** is the smoothening parameter and quantifies the amount of material removed. The larger **wₑₓₜ**, the more predominant **shape** will be.

In [14]:
shape.observe(update_texts, names='value')
N.observe(update_texts, names='value')
wext.observe(update_texts, names='value')

N.observe(update_wext_max, names='value')
wext.observe(update_N_min, names='value')

shape.observe(update_downloadbutton, names='value')
N.observe(update_downloadbutton, names='value')
wext.observe(update_downloadbutton, names='value')

display(widgets.interactive(update_image, shape=shape, N=N, r=wext))

interactive(children=(Dropdown(description='shape', options=('armchair', 'round', 'zigzag'), value='armchair')…

## Numerical informations

In the following, a summary of numerical informations is displayed.

In [15]:
display(
    widgets.VBox([
        a_ratio,
        z_ratio,
        tau_around_0p00_ev,
        tau_around_0p01_ev,
        tau_around_0p10_ev,
    ]),
)

## Download

Here you can download informations about the junction.
* **Summary** contains informations about the junction's geometry, edge structure and preserved conductance at different energy windows.
* **Conductance** contains the conductance [$G_0$] of the junction at different energies [$\mathrm{eV}$].
* **Vertices** contains a list of the junction's vertices' positions [$\sqrt{3}a$], as well as the local density of states (LDOS) at each vertex.
* **Edges** contains a list of the junction's edges, as well as the probability currents flowing through each edge.

In [16]:
display(
    widgets.HBox([
    html_button_summary,
    html_button_conductance,
    html_button_vertices,
    html_button_edges,
    ]))